In [5]:
import os
import sys
import gains as genetic
from gains.salt_generator import *
import pandas as pd
import rdkit
import pickle
from math import exp
import numpy as np
import random
import unittest
import datetime
import sys
import sklearn
import salty

from numpy import array, all
from rdkit.Chem import Draw
from rdkit import RDConfig
from rdkit.Chem import FragmentCatalog
from rdkit import RDConfig
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import Chem
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import ShowMol
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.ML.Descriptors.MoleculeDescriptors import\
    MolecularDescriptorCalculator as calculator

%matplotlib inline

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]   

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  

In [6]:
target = 1000
model_ID = "density_m3"
generate_solvent(target, model_ID, sim_bounds=[0.1, 1], heavy_atom_limit=1e8, hits=1)

/Users/wesleybeckner/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.19.1 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[S+](C)(CC)(CC)	0.9515	starting structure
CC[S+](C)C(C)NC	0.99671	add_rdkit_fragment
number of atoms: 	8
mutation attempts: 	1
with density: 		996.71 (kg/m)
similarity score:       0.460
molecular relative: 	2-hydroxy-N-(2-hydroxyethyl)-N-methylethanaminium



,Salt ID,Salt Smiles,Cation Heavy Atoms,Tanimoto Similarity Score,Molecular Relative,Anion,Model Density,MD Density,Error
0,C01_A01,CC[S+](C)C(C)NC.N#CN=C=[N-],8,0.46,2-hydroxy-N-(2-hydroxyethyl)-N-methylethanaminium,dicyanoazanide,996.71,NaN,NaN


In [7]:
salty.check_name("phosphate")

'[O-]P(=O)([O-])[O-]'

In [19]:
parent_candidates = eval(genetic.load_data("{}_summary.csv".
                             format(model_ID)).loc[1][1])
for i in range(len(parent_candidates)):
    print(salty.check_name(parent_candidates[i]))

1-butyl-2-methylpyridinium
1,2-dimethyl-3-propylimidazolium
1-butyl-2,3-dimethylimidazolium
1-methyl-3-propylimidazolium
2-hydroxyethanaminium
1-phenyl-3,5-dimethyl-2-pentylpyrazolium
1-phenyl-3,5-dimethyl-2-hexylpyrazolium
1-methyl-3-octyl-1H-imidazol-3-ium
1-butyl-3-methylimidazolium
1-ethyl-3-methylimidazolium
3-hexyl-1-methylimidazolium
1-butyl-3-methylpyridinium
1-butylpyridinium
1-propylpyridinium
N-octyl-3-methylpyridinium
1-decyl-3-methylimidazolium
1-ethylpyridinium
1-hexylpyridinium
3-methyl-1-pentylimidazolium
1-(2-hydroxyethyl)-3-methylimidazolium
1-octylpyridinium
N,N-dimethylformamide
2-hydroxy-N-(2-hydroxyethyl)-N-methylethanaminium
1-(2-methoxyethyl)-3-methyl-imidazolium
N,N-diethyl-2-methoxy-N-methylethan-1-aminium
N,N,N-triethyl-N-hexylammonium
N,N,N-triethyl-N-octylammonium
tetraethylammonium
1-(2-hydroxyethyl)-2,3-dimethylimidazolium
hexyl(2-hydroxyethyl)dimethylammonium
N-methylpyrrolidone
1-hexyloxymethyl-3-methylimidazolium
3-(2-aminoethyl)-1-methyl-1H-imidazoliu

In [19]:
true

21    1290.87
Name: Model Density, dtype: float64

In [24]:
out = pd.DataFrame()
for j in np.arange(60,100,10):
    df = pd.read_csv("../../designer_molecules/pdb_files/model_3/1300kgm3/%s/salt_log.csv" % j) #or salt_candidates
    inner = pd.DataFrame()
    for i in range(1,26):
        if i < 10:
            CAT_ID = "C0%s" % i
            AN_ID = "A0%s" % i
        else:
            CAT_ID = "C%s" % i
            AN_ID = "A%s" % i
        Salt_ID = CAT_ID + "_" + AN_ID
        try:
            mean = np.round(pd.read_csv("../../designer_molecules/pdb_files/model_3/1300kgm3/%s/%s.dens" % (j, Salt_ID)).describe().loc["mean"][1])
            std = np.round(pd.read_csv("../../designer_molecules/pdb_files/model_3/1300kgm3/%s/%s.dens" % (j, Salt_ID)).describe().loc["std"][1])
            value = ("{}{}{}".format(mean," +/- ", std))
            true = df.loc[df["Salt ID"] == Salt_ID, "Model Density"]
            difference = (true - mean)/true*100
            df.loc[df["Salt ID"] == Salt_ID, "MD Density"] = value
            df.loc[df["Salt ID"] == Salt_ID, "Error"] = difference #this is % Difference in old version
        except:
            pass
    df.dropna(inplace=True)
    out = pd.concat([df, out], axis=0) 
out.reset_index(drop=True, inplace=True)
out

,Salt ID,Salt Smiles,Cation Heavy Atoms,Tanimoto Similarity Score,Molecular Relative,Anion,Model Density,MD Density,Error
0,C04_A04,CCCCCn1c(C)cc(C)[n+]1-c1cccc(O)c1.O=C([O-])C(F...,19,0.911271,"1-phenyl-3,5-dimethyl-2-pentylpyrazolium",trifluoroacetate,1297.11,1292.0 +/- 41.0,0.393953
1,C08_A08,CCCC(CC)[n+]1ccn(C)c1.Cl[Fe-](Cl)(Cl)Cl,12,0.915179,1-cyclopentyl-3-butylimidazolium,tetrachloroferrate,1306.20,1273.0 +/- 27.0,2.541724
2,C09_A09,C[n+]1ccn(CCCCCCCCCCC#N)c1.O=S(=O)([N-]S(=O)(=...,18,0.948571,1-methyl-3-octyl-1H-imidazol-3-ium,bis(trifluoromethylsulfonyl)imide,1292.21,1194.0 +/- 34.0,7.600158
3,C11_A11,CC[n+]1ccn(CC#N)c1.CCOS(=O)(=O)[O-],10,0.961240,3-(cyanomethyl)-1-methyl-1H-imidazol-3-ium,ethylsulfate,1291.20,1225.0 +/- 35.0,5.127014
4,C15_A15,CSCCCCCCCC[n+]1ccccc1.O=S(=O)(F)[N-]S(=O)(=O)F,16,0.947826,1-octylpyridinium,bis(fluorosulfonyl)amide,1301.89,1259.0 +/- 41.0,3.294441
5,C18_A18,CCCCC(CC)C[n+]1cc(C)cc(C)c1CC.O=S(=O)([O-])C(F...,18,0.932347,"1-hexyl-3,5-dimethylpyridinium","1,1,2-trifluoro-2-(perfluoroethoxy)ethanesulfo...",1308.38,1179.0 +/- 35.0,9.888564
6,C19_A19,CC(N)CCCCn1cc[n+](CCC#N)c1.[O-][Cl+3]([O-])([O...,16,0.925373,3-(2-cyanoethyl)-1-hexyl-1H-imidazolium,perchlorate,1291.67,1136.0 +/- 43.0,12.051840
7,C22_A22,C#CCCCn1cc[n+](C)c1.O=C(O)c1cccc(S(=O)(=O)[O-])c1,11,0.924138,1-butyl-3-methylimidazolium,3-sulfobenzoate,1290.87,1323.0 +/- 49.0,-2.489019
8,C01_A01,CCCC(CC)[n+]1cccc(C#N)c1.O=C(O)c1cccc(S(=O)(=O...,14,0.841463,3-cyano-1-octylpyridinium,3-sulfobenzoate,1293.30,1174.0 +/- 55.0,9.224465
9,C02_A02,CC(CCCC(F)(F)F)n1cc[n+](C)c1.O=S(=O)([O-])C(F)...,15,0.829960,1-cyclopentyl-3-butylimidazolium,"1,1,2,3,3,3-hexafluoro-1-propanesulfonate",1290.59,1394.0 +/- 43.0,-8.012614


In [ ]:
out.drop([23,58], inplace=True)

In [ ]:
driver = webdriver.Chrome()
driver.get("http://www.chemspider.com/")
out.reset_index(inplace=True)
results=[]

In [ ]:
###This cell checks that entries are not existent in the world
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

for i in range(52,out.shape[0]):
    ion = out["Salt Smiles"][i].split(".")[0]
    search = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00_qs_query\"]")
    search.clear()
    search.send_keys(ion)
    submit = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00_search_btn\"]")
    submit.click()
    time.sleep(5)
    try:
        name = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00_ContentSection"\
                                            "_ContentPlaceHolder1_RecordViewDetails"\
                                            "_rptDetailsView_ctl00_WrapTitle\"]")
        results.append([ion, name.text])
        print(ion, name.text)
        time.sleep(3)
    except:
        results.append([ion, "new molecule"])
        print("{}\t{}".format("name not found for:",ion))

In [20]:
b = pd.DataFrame(a)
b.to_csv("existance_results.csv",index=False, indicator=True)

NameError: name 'a' is not defined

In [ ]:
results
a = np.array(results)
a[:,1]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as lin
poop = np.abs(out["% Difference"].values)
with plt.style.context('seaborn-whitegrid'):
    fig=plt.figure(figsize=(5,5), dpi=300)
    ax = fig.add_subplot(111)
    ax.plot(out["Tanimoto Similarity Score"], poop, linestyle="", marker="*", label=None)
    #ax.plot([0,X0],[Y0,0], label="Best fit R-squared: 0.25")
    ax.set_xlim(0.4,1)
    ax.set_ylim(0,11)
    #ax.legend(bbox_to_anchor=(1,1))
    ax.set_ylabel("% Error")
    ax.set_xlabel("Similarity Score")
    ax.grid()

In [ ]:
df = genetic.load_data("cationInfo.csv")
parent_candidates = df["smiles"].unique()
to_plot=[]
for i in range(out.shape[0]):
    smi = out.iloc[i]["Salt Smiles"].split(".")[0]
    mol = Chem.MolFromSmiles(smi)
    chro = genetic.Chromosome(smi, 0)
    score, index = molecular_simularity(chro, parent_candidates)
    score.append(out.iloc[i]["% Difference"])
    to_plot.append(score)

In [ ]:
points = np.array(to_plot)
sim_scores = []
for i in range(points.shape[1]-1):
    sim_scores.append(points[:,i])

In [ ]:
metrics = ["Tanimoto, 0.21", "Dice, 0.22", "Cosine, 0.21", "Sokal, 0.20", 
           "Kulczynski, 0.20", "McConnaughey, 0.20"]
# metrics = ["Tanimoto", "Dice", "Cosine", "Sokal", 
#            "Kulczynski", "McConnaughey"]

import matplotlib.pyplot as plt
with plt.style.context('seaborn-whitegrid'):
    fig=plt.figure(figsize=(5,5), dpi=300)
    ax = fig.add_subplot(111)
    for i in range(len(sim_scores)):
        X=sim_scores[i].reshape(-1,1)
        Y=np.abs(out["% Difference"]).values.reshape(-1,1)
        model = lin()
        model.fit(X,Y)
        if model.coef_ < 0:
            Y0=model.intercept_
            X0=abs(model.intercept_/model.coef_)
        else:
            print("pos corr")
        print(model.coef_, model.intercept_, model.score(X,Y))
        for j in range(len(sim_scores[0])):
        
            if j == 0:
                ax.plot(sim_scores[i][j], np.abs(out.iloc[j]["% Difference"]), 
                        linestyle="", marker="*", c=tableau20[i*2],
                       label=metrics[i])
                ax.plot([0,X0],[Y0,0])
    
            else:
                ax.plot(sim_scores[i][j], np.abs(out.iloc[j]["% Difference"]), 
                        linestyle="", marker="*", c=tableau20[i*2])
    ax.set_xlim(0.2,1)
    ax.set_ylim(0,11)
    ax.set_ylabel("% Error")
    ax.set_xlabel("Similarity Score")
    ax.legend(bbox_to_anchor=(1,1))
    
    ax.grid()
#fig.savefig(filename='poster_images/NN_Test_Dataset_All_Salts.eps', 
#                        bbox_inches='tight', format='eps') 

In [ ]:
# salts = salty.load_data("salts_with_smiles.csv")
# new_df = pd.concat([salts["name-cation"], salts["name-anion"], salts["Temperature, K"],\
#                     salts["Pressure, kPa"], salts["Specific density, kg/m<SUP>3</SUP>"]],\
#                    axis = 1)
# cationDescriptors = salty.load_data("cationDescriptors.csv")
# cationDescriptors.columns = [str(col) + '-cation' for col in cationDescriptors.columns]
# anionDescriptors = salty.load_data("anionDescriptors.csv")
# anionDescriptors.columns = [str(col) + '-anion' for col in anionDescriptors.columns]
# new_df = pd.merge(cationDescriptors, new_df, on="name-cation", how="right")
# new_df = pd.merge(anionDescriptors, new_df, on="name-anion", how="right")
# new_df.dropna(inplace=True) 

df = genetic.load_data("cationInfo.csv")
parent_candidates = df["smiles"].unique()

mol = Chem.MolFromPDBFile("pdb_files/model_3/90/C02.pdb")
Draw.MolToMPL(mol)
smi = Chem.MolToSmiles(mol)
print(smi)
best = genetic.Chromosome(smi, 0)
score, index = molecular_simularity(best, parent_candidates)
# print(parent_candidates[index])
print(salty.check_name(parent_candidates[index[0]]))
print(np.round(score, decimals=2))
print(np.round(np.average(score), decimals=3))
print(index)

print("{}\t{}".format("cation heavy atoms: ", mol.GetNumAtoms()))
mol = Chem.MolFromPDBFile("pdb_files/model_3/90/A02.pdb")
smi = Chem.MolToSmiles(mol)
print("{}\t{}".format("anion heavy atoms: ", mol.GetNumAtoms()))
print(smi)
# print(salty.check_name(smi))
Draw.MolToMPL(mol)

# smi = salty.check_name("methyl sulfate")
# print(smi)
# mol = Chem.MolFromSmiles(smi)
# Draw.MolToMPL(mol)

In [ ]:
parent_candidates[129]

In [ ]:
cation = Chem.MolFromSmiles(salty.check_name("1-butyl-1-methylpyrrolidinium"))
cation = Chem.AddHs(cation)
Chem.EmbedMolecule(cation, Chem.ETKDG())
Chem.UFFOptimizeMolecule(cation)
rdkit.Chem.rdmolfiles.MolToPDBFile(cation, "pdb_files/prototype/BMP")


anion = Chem.MolFromSmiles(salty.check_name("bis[(trifluoromethyl)sulfonyl]imide"))
anion = Chem.AddHs(anion)
Chem.EmbedMolecule(anion, Chem.ETKDG())
Chem.UFFOptimizeMolecule(anion)
rdkit.Chem.rdmolfiles.MolToPDBFile(anion, "pdb_files/prototype/TF2")

In [ ]:
def guess_password(target):
    startTime = datetime.datetime.now()

    def fnGetFitness(genes):
        return get_fitness(anion, genes, target)

    def fnDisplay(candidate, mutation):
        display(candidate, mutation, startTime)

    def fnShowIon(genes, target, mutation_attempts):
        show_ion(genes, target, mutation_attempts)

    optimalFitness = 0.99
    best = genetic.get_best(fnGetFitness,\
        optimalFitness, geneSet, fnDisplay,\
                    fnShowIon, target, parent_candidates)
    return best
    
def display(candidate, mutation, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}".format(
    candidate.Genes, candidate.Fitness, mutation))
    
def get_fitness(anion, genes, target):
    cation = Chem.MolFromSmiles(genes)
    model = genetic.load_data("density_nn_model.sav", pickleFile=True)
    deslist = genetic.load_data("density_nn_model_descriptors.csv")
    feature_vector=[]
    
    for item in deslist:

        if "anion" in item:
            with genetic.suppress_stdout_stderr():
                feature_vector.append(calculator([item.partition('-')\
                    [0]]).CalcDescriptors(anion)[0])
        elif "cation" in item:
            with genetic.suppress_stdout_stderr():
                feature_vector.append(calculator([item.partition('-')\
                [0]]).CalcDescriptors(cation)[0])          
        elif "Temperature_K" in item:
            feature_vector.append(298.15)
        elif "Pressure_kPa" in item:
            feature_vector.append(101.325)
        else:
            print("unknown descriptor in list: %s" % item)
    features_normalized = (feature_vector-deslist.iloc[0].values)/deslist.iloc[1].values
    prediction = exp(model.predict(np.array(features_normalized).reshape(1,-1))[0])
    error = abs((prediction-target)/target)
    
    return 1-error

def show_ion(genes, target, mutation_attempts):
    print("{}\t{}".format("mutation attempts: ", mutation_attempts))
    print("within 1%% of target density: %s (kg/m) " % target)
    

geneSet = genetic.generate_geneset()

df = genetic.load_data("cationInfo.csv")
# df = df.loc[df["name"].str.contains("imid", case=False)]
# df = df.loc[~df["name"].str.contains("phenyl", case=False)]
# df = df.loc[~df["name"].str.contains("benzyl", case=False)]
# df = df.loc[~df["name"].str.contains("azido", case=False)]
# df = df.loc[~df["name"].str.contains("cyan", case=False)]
# df = df.loc[~df["name"].str.contains("benz", case=False)]
# df = df.loc[~df["name"].str.contains("cyclo", case=False)]
# df = df.loc[~df["name"].str.contains("sulf", case=False)]
# df = df.loc[~df["name"].str.contains("azepinium", case=False)]
parent_candidates = df['smiles'].unique()


df = genetic.load_data("anionInfo.csv")
df = df['smiles'].unique()
ohPickMe = random.sample(range(df.shape[0]),1)
anion = Chem.MolFromSmiles(df[ohPickMe[0]])
print(df[ohPickMe[0]])
img = Draw.MolToMPL(anion)
target = random.sample(range(800,1500),1)[0]

print(target)
best = guess_password(target)
img = Draw.MolToMPL(best.Mol)

In [ ]:
smiles = "CCCCCCc1c(SC)[n+](C)c(S)n1CCCPC"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
Chem.EmbedMolecule(mol, Chem.ETKDG())
Chem.UFFOptimizeMolecule(mol)
a = Draw.MolToMPL(mol)
rdkit.Chem.rdmolfiles.MolToPDBFile(mol, "test", confId=-2)

In [ ]:
def guess_password(target):
    startTime = datetime.datetime.now()

    def fnGetFitness(genes):
        return get_fitness(genes, target)

    def fnDisplay(candidate, mutation):
        display(candidate, mutation, startTime)

    def fnShowIon(genes, target, mutation_attempts):
        show_ion(genes, target, mutation_attempts)

    optimalFitness = get_fitness(target, target)
    best = genetic.get_best(fnGetFitness,\
        optimalFitness, geneSet, fnDisplay,\
                    fnShowIon, target, parent_candidates)
    
def display(candidate, mutation, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}\t{}".format(
    candidate.Genes, candidate.Fitness, mutation, timeDiff))
    
def get_fitness(genes, target):
    ms = [Chem.MolFromSmiles(target), Chem.MolFromSmiles(genes)]
    fps = [FingerprintMols.FingerprintMol(x) for x in ms]
    return DataStructs.FingerprintSimilarity(fps[0],fps[1])

def show_ion(genes, target, mutation_attempts):
    mol = Chem.MolFromSmiles(target)
    print("{}\t{}".format("number of atoms: ", mol.GetNumAtoms()))
    print("{}\t{}".format("mutation attempts: ", mutation_attempts))
    

geneSet = genetic.generate_geneset()

df = genetic.load_data("cationInfo.csv")
df = df.loc[df["name"].str.contains("imid", case=False)]
df = df.loc[~df["name"].str.contains("phenyl", case=False)]
df = df.loc[~df["name"].str.contains("benzyl", case=False)]
df = df.loc[~df["name"].str.contains("azido", case=False)]
df = df.loc[~df["name"].str.contains("cyan", case=False)]
df = df.loc[~df["name"].str.contains("benz", case=False)]
df = df.loc[~df["name"].str.contains("cyclo", case=False)]
df = df.loc[~df["name"].str.contains("sulf", case=False)]
df = df.loc[~df["name"].str.contains("azepinium", case=False)]
parent_candidates = df['smiles'].unique()

df = parent_candidates
ohPickMe = random.sample(range(df.shape[0]),1)
target = df[ohPickMe[0]]
guess_password(target)
img = Draw.MolToMPL(Chem.MolFromSmiles(target))

In [ ]:
from rdkit.Chem import MolFromSmiles as mol
from rdkit.Chem.Draw import MolToMPL as img
a = img(mol(target))

In [ ]:
geneSet.GetEntryDescription(random.sample(range(geneSet.GetNumEntries()), 1)[0])

In [ ]:
Chromosome("C1[n+]ccn1",0)

In [ ]:
Chromosome(geneSet.GetEntryDescription(
    random.sample(range(geneSet.GetNumEntries()), 1)[0]), 0)

In [ ]:
atoms = [6, 7]
fName = os.path.join(RDConfig.RDDataDir, 'FunctionalGroups.txt')
rdkitFrags = FragmentCatalog.FragCatParams(1, 5, fName)
customFrags = FragmentCatalog.FragCatalog(rdkitFrags)
fcgen = FragmentCatalog.FragCatGenerator()
m = Chem.MolFromSmiles(check_name("1-benzyl-3-methyl-1H-imidazol-3-ium"))
fcgen.AddFragsFromMol(m, customFrags)

geneSet = customFrags
newGene = Chromosome(geneSet.GetEntryDescription(
    random.sample(range(geneSet.GetNumEntries()), 1)[0]), 0)
oldGene = oldGene + newGene.Mol.GetNumAtoms()
combined = Chem.EditableMol(Chem.CombineMols(newGene.Mol,
                            childGenes.Mol))
combined.AddBond(0, oldGene, order=Chem.rdchem.BondType.SINGLE)
childGenes = combined.GetMol()

childGenes = Chromosome(Chem.MolToSmiles(childGenes), 0)

In [ ]:
df = load_data("cationInfo.csv")
df = df.loc[df["name"].str.contains("imid", case=False)]
df = df.loc[~df["name"].str.contains("phenyl", case=False)]
df = df.loc[~df["name"].str.contains("benzyl", case=False)]
df = df.loc[~df["name"].str.contains("azido", case=False)]
df = df.loc[~df["name"].str.contains("cyan", case=False)]
df = df.loc[~df["name"].str.contains("benz", case=False)]
df = df.loc[~df["name"].str.contains("cyclo", case=False)]
df = df.loc[~df["name"].str.contains("sulf", case=False)]
df = df.loc[~df["name"].str.contains("azepinium", case=False)]
df = df['smiles'].unique()
for i in range(df.shape[0]):
    Draw.MolToMPL(Chem.MolFromSmiles(df[i]))

In [ ]:
from salty import check_name
df = load_data("cationInfo.csv")
df = df.loc[df["name"].str.contains("imid", case=False)]
df = df.loc[~df["name"].str.contains("phenyl", case=False)]
df = df.loc[~df["name"].str.contains("benzyl", case=False)]
df = df.loc[~df["name"].str.contains("azido", case=False)]
df = df.loc[~df["name"].str.contains("cyan", case=False)]
df = df.loc[~df["name"].str.contains("benz", case=False)]
df = df.loc[~df["name"].str.contains("cyclo", case=False)]
df = df.loc[~df["name"].str.contains("sulf", case=False)]

df = df['smiles'].unique()
df


In [ ]:
def remove_custom_fragment(childGenes, GeneSet, oldGene):
    geneSet = GeneSet.CustomFrags
    newGene = Chromosome(geneSet.GetEntryDescription(
        random.sample(range(geneSet.GetNumEntries()), 1)[0]), 0)
    try:
        truncate = Chem.DeleteSubstructs(childGenes.Mol, newGene.Mol)
        childGenes = truncate
        childGenes = Chromosome(Chem.MolToSmiles(childGenes), 0)
        return childGenes
    except BaseException:
        return 0
img = Draw.MolToMPL(Chem.MolFromSmiles("CNN(CCCCCN1C=C[NH+]=C1)CCC(F)CCCCCCN=O"))

In [ ]:
import salty
salty.check_name("c1cc(cc(c1)S(=O)(=O)[O-])C(=O)O")

In [ ]:
geneSet = genetic.generate_geneset()
df = genetic.load_data("saltInfo.csv")
df = df.loc[df["cation_name"].str.contains("imid", case=False)]
df = df['cation_SMILES'].unique()
ohPickMe = random.sample(range(df.shape[0]),2)
target = Chem.MolFromSmiles(df[ohPickMe[0]])
candidate = Chem.MolFromSmiles(df[ohPickMe[1]])
%matplotlib inline
SimilarityMaps.GetSimilarityMapForFingerprint(candidate,\
        target,SimilarityMaps.GetMorganFingerprint)